In [750]:
import numpy as np

In [751]:
import pandas as pd

In [879]:
users=pd.read_csv('/home/HDUSER/users.csv')

In [880]:
news=pd.read_csv('/home/HDUSER/news.csv')

In [881]:
users.shape

(10, 7)

In [882]:
users.head()

,user_id,tweet_id,tweet,summary,date,retweets,favorites
0,xyz123,22,president trump visited china. Hope will have a great success between us and china. #ChinaVisit #Trump,president trump visit china success US,09/11/17,78,56
1,yudi,14,"Earthquake magnitude 8.5 in North India, epicentre New Delhi. #Earthquake, #BeSafe, #DelhiShakes",Earthquake Delhi 8.5 North India,05/11/17,2,4
2,abc123,11,"Roughly speaking, level of pollution in some parts of Delhi is like smoking 50 cigarettes a day. #DelhiSmog",Pollution Delhi Smoking Cigarettes Smog,09/11/17,32,29
3,alpha,10,Halloween party for kids in USA organized. Chocolates and sweets to be distributed,Halloween party kids USA chocolates sweets,10/11/17,42,41
4,beta,1,prisoners can't move freely. None of us have ever seen him. we are just told he is lodged here: Rahul,Rohtak jail prisoner out on bail|prisoners freely lodged,09/11/17,75,21


In [883]:
news.shape

(20, 4)

In [884]:
news.head()

,news_id,news_link,news_summary,date
0,10.0,https://economictimes.indiatimes.com/news/politics-and-nation/delhi-smog-air-quality-likely-to-get-worse-today/articleshow/61568222.cms,To encourage use of public transport during Odd Even Delhi govt to allow free travel for commuters in all DTC and Cluster buses from 13-17 November NGT asked DTC on the number of its unused buses in its workshops The National Green Tribunal asks CPCB DPCC and Delhi Govt why didn't you implement it earlier Nothing has been done from your Delhi govt end since the past one year SAFAR has forecast increase in the concentration of fine particles of pollutants on Thursday Madhavan Rajeevan secretary ministry of earth sciences tweeted Delhi smog is not localised as seen in satellite image I call for cost effective measures since management of air pollution requires sustained actions over a long period of time to be effective,9112017.0
1,11.0,http://www.firstpost.com/world/donald-trumps-china-visit-xi-jinpings-political-star-is-ascendant-north-korea-and-trade-dominate-talks-4201407.html,On the anniversary of his surprise victory in the 2016 presidential election US president Donald Trump on Wednesday reached Beijing on his first visit to China Time magazine established the significance of Trump’s China visit in just one simple headline President Trump meets world's most powerful man in Beijing The US president has long been considered the world's most powerful man North Korea crisisNorth Korea crisis which has been lingering since April as well as bilateral trade was always expected to be on top of Trump’s agenda China is one of the two neighbouring countries with which North Korea has a close relationship The other being Russia After his interaction with Xi Trump said Our meeting this Thursday morning in front of my representatives and your representatives was excellent discussing North Korea.,9112017.0
2,12.0,https://timesofindia.indiatimes.com/pm-modi-meets-new-zealand-pm-john-key/liveblog/55062708.cms,India to talk on its NSG bid with New Zealand New Zealand was one of the countries that took the stand at the last NSG plenary in South Korea in June that no exception can be made in the case of India a non-NPT country while considering its membership bid of the elite group that regulates trade in atomic material At the plenary despite strong US support China had blocked India's bid on the ground that it was a not a signatory to the Nuclear Non-Proliferation Treaty External affairs ministry spokesperson Vikas Swarup said India will tell New Zealand it had all the credentials to become a member of the Nuclear Suppliers Group and that it would strengthen the NPT regime,NaN
3,13.0,https://www.ndtv.com/tamil-nadu-news/rs-1-430-crore-in-undeclared-wealth-found-in-raids-on-sasikala-jaya-tv-1775019,"The raids also targetted Jaya TV which turned a bitter critic of the EPS government since it joined hands with OPS Income Tax officials say they have found Rs 1,430 crore in undeclared income in raids on Jaya TV and family members of jailed AIADMK party leader VK Sasikala including brother VK Divakaran Authorities also say Rs 7 crore in cash Rs 5 crore worth of gold and a huge collection of diamond jewellery the value of which is yet to be assessed have been recovered Though the key target was the family that operates Jaya TV the other nine groups we now believe all had some kind of monetary transaction with them They could potentially bring down the EPS government if the court quashes the Speaker's order The Income Tax department however denies any political links and says the raids were part of an operation called clean money drive",NaN
4,14.0,http://www.thehindubusinessline.com/news/national/gujarat-assembly-elections-rahul-gandhi-narendra-modi-kheda/article9895724.ece,Congress Vice-President Rahul Gandhi continued his offensive on Prime Minister Narendra Modi during his poll campaigning in Central Gujarat on Monday In a veiled attack on the Prime Minister’s monthly

In [885]:
t1=users['summary']

In [886]:
t2=news['news_summary']

In [887]:
t1=t1.str.lower()

In [888]:
t2=t2.str.lower()

In [889]:
t2

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                            to encourage use of public transport during odd even delhi govt to allow free travel for commuters in all dtc and cluster buses from 13-17 november ngt asked dtc on the number of its unused buses in its workshops the national green tribunal asks cpcb dpcc and delhi govt why didn't you implement it earlier nothing has been done from your delhi govt end since the past one year safar has forecast increase in the concentration of fine particles of pollutants on thursday madhavan rajeevan secretary ministry of earth sciences tweeted delhi

In [890]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [891]:
t1

0                                               president trump visit china success us
1                                                     earthquake delhi 8.5 north india
2                                              pollution delhi smoking cigarettes smog
3                                           halloween party kids usa chocolates sweets
4                             rohtak jail prisoner out on bail|prisoners freely lodged
5                                 pm narendra modi meets new zealand pm jacinda ardern
6    properties dawoodibrahim relatives auctioned imc chamber commerce industry mumbai
7                           court bihar bettiah fir farooq abdullah treason lawyer pok
8     incometax unaccounted income 1400 crore unearthed raids sasikala jaya tv chennai
9                         congress power mann ki baat government rahul gandhi visnagar
Name: summary, dtype: object

In [892]:
from nltk.tokenize import word_tokenize

In [893]:
import nltk

In [894]:
import string
import math

In [895]:
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

def term_frequency(term, tokenized_document):
    return tokenized_document.count(term)

def sublinear_term_frequency(term, tokenized_document):
    count = tokenized_document.count(term)
    if count == 0:
        return 0
    return 1 + math.log(count)

def augmented_term_frequency(term, tokenized_document):
    max_count = max([term_frequency(t, tokenized_document) for t in tokenized_document])
    return (0.5 + ((0.5 * term_frequency(term, tokenized_document))/max_count))

def inverse_document_frequencies(tokenized_documents):
    idf_values = {}
    all_tokens_set = set([item for sublist in tokenized_documents for item in sublist])
    for tkn in all_tokens_set:
        contains_token = map(lambda doc: tkn in doc, tokenized_documents)
        idf_values[tkn] = 1 + math.log(len(tokenized_documents)/(sum(contains_token)))
    return idf_values

def tfidf(documents):
    tokenized_documents = [tokenize(d) for d in documents]
    idf = inverse_document_frequencies(tokenized_documents)
    tfidf_documents = []
    for document in tokenized_documents:
        doc_tfidf = []
        for term in idf.keys():
            tf = sublinear_term_frequency(term, document)
            doc_tfidf.append(tf * idf[term])
        tfidf_documents.append(doc_tfidf)
    return tfidf_documents

In [896]:
tokenize = lambda doc: doc.lower().split(" ")

In [897]:
sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
sklearn_representation = sklearn_tfidf.fit_transform(t2)

In [898]:
def cosine_similarity(vector1, vector2):
    dot_product = sum(p*q for p,q in zip(vector1, vector2))
    magnitude = math.sqrt(sum([val**2 for val in vector1])) * math.sqrt(sum([val**2 for val in vector2]))
    if not magnitude:
        return 0
    return dot_product/magnitude

In [899]:
def normalize(word_vec):
    norm=np.linalg.norm(word_vec)
    if norm == 0: 
       return word_vec
    return word_vec/norm

In [900]:
tfidf_representation = tfidf(t2)

In [901]:
sklearn_tfidf1 = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
sklearn_representation1 = sklearn_tfidf1.fit_transform(t1)

In [902]:
tfidf_representation2=tfidf(t1)

In [903]:
our_tfidf_comparisons = []
for count_0, doc_0 in enumerate(tfidf_representation):
    for count_1, doc_1 in enumerate(tfidf_representation2):
        our_tfidf_comparisons.append((cosine_similarity(doc_0, doc_1), count_0, count_1))


In [904]:
skl_tfidf_comparisons = []
for count_0, doc_0 in enumerate(sklearn_representation.toarray()):
    for count_1, doc_1 in enumerate(sklearn_representation1.toarray()):
        skl_tfidf_comparisons.append((cosine_similarity(doc_0, doc_1), count_0, count_1))

In [905]:
for x in zip(sorted(our_tfidf_comparisons, reverse = True), sorted(skl_tfidf_comparisons, reverse = True)):
    print(x)

((0.14690771929142468, 12, 2), (0.13107672416749572, 17, 7))
((0.14054988436074017, 2, 1), (0.12225387336640035, 19, 1))
((0.1342388173465582, 6, 5), (0.10923776225206806, 13, 8))
((0.125622756841313, 11, 0), (0.10302541130581828, 9, 0))
((0.1205026707627734, 0, 8), (0.10211889405732671, 7, 3))
((0.09764800197069087, 8, 9), (0.099714695006528145, 16, 9))
((0.09558484752162044, 17, 6), (0.096123332789655472, 5, 0))
((0.09317170921221629, 4, 3), (0.094548430635718453, 10, 8))
((0.09041933211411464, 14, 5), (0.094378583253301382, 14, 1))
((0.08490328414236645, 13, 0), (0.09404308003255428, 6, 8))
((0.08360888387028674, 16, 8), (0.090636355337967836, 8, 8))
((0.08308952734100832, 8, 5), (0.087966752893136213, 16, 8))
((0.08285530634857692, 14, 3), (0.084281939010415141, 7, 4))
((0.082388880732608, 7, 3), (0.078611028373384489, 3, 6))
((0.07916374054914384, 12, 0), (0.076535118600517149, 3, 8))
((0.07860137037503546, 1, 2), (0.075710528166932634, 17, 8))
((0.07563415750023213, 6, 8), (0.072

In [906]:
t1.head()

0                      president trump visit china success us
1                            earthquake delhi 8.5 north india
2                     pollution delhi smoking cigarettes smog
3                  halloween party kids usa chocolates sweets
4    rohtak jail prisoner out on bail|prisoners freely lodged
Name: summary, dtype: object

In [907]:
t2.head()

0                                                                                                                                                                                                                                                                                         to encourage use of public transport during odd even delhi govt to allow free travel for commuters in all dtc and cluster buses from 13-17 november ngt asked dtc on the number of its unused buses in its workshops the national green tribunal asks cpcb dpcc and delhi govt why didn't you implement it earlier nothing has been done from your delhi govt end since the past one year safar has forecast increase in the concentration of fine particles of pollutants on thursday madhavan rajeevan secretary ministry of earth sciences tweeted delhi smog is not localised as seen in satellite image i call for cost effective measures since management of air pollution requires sustained actions over a long period of time to be 

In [908]:
sklearn_representation

<20x982 sparse matrix of type '<class 'numpy.float64'>'
	with 1394 stored elements in Compressed Sparse Row format>

In [909]:
sklearn_representation1

<10x75 sparse matrix of type '<class 'numpy.float64'>'
	with 76 stored elements in Compressed Sparse Row format>

In [910]:
print(sklearn_representation)

  (0, 884)	0.0611916286585
  (0, 292)	0.105402821144
  (0, 916)	0.0871183958378
  (0, 628)	0.111185741357
  (0, 703)	0.105402821144
  (0, 895)	0.105402821144
  (0, 276)	0.0871183958378
  (0, 626)	0.105402821144
  (0, 303)	0.0764226926865
  (0, 239)	0.207890136738
  (0, 382)	0.221199655713
  (0, 59)	0.105402821144
  (0, 347)	0.105402821144
  (0, 896)	0.0871183958378
  (0, 340)	0.071364633403
  (0, 188)	0.105402821144
  (0, 432)	0.085399567812
  (0, 58)	0.0629477003268
  (0, 272)	0.178462489443
  (0, 68)	0.0469542015347
  (0, 178)	0.105402821144
  (0, 141)	0.178462489443
  (0, 350)	0.0855878199774
  (0, 14)	0.105402821144
  (0, 616)	0.105402821144
  :	:
  (18, 727)	0.160453018723
  (18, 802)	0.160453018723
  (18, 352)	0.160453018723
  (18, 832)	0.160453018723
  (19, 340)	0.105128681828
  (19, 68)	0.0691691819635
  (19, 865)	0.0691691819635
  (19, 415)	0.126081145039
  (19, 963)	0.217291511766
  (19, 908)	0.217291511766
  (19, 709)	0.217291511766
  (19, 948)	0.157004738605
  (19, 145)	0.1

In [911]:
print(sklearn_representation1)

  (0, 53)	0.408248290464
  (0, 66)	0.408248290464
  (0, 72)	0.408248290464
  (0, 11)	0.408248290464
  (0, 63)	0.408248290464
  (0, 70)	0.408248290464
  (1, 20)	0.465026284587
  (1, 19)	0.367426480502
  (1, 1)	0.465026284587
  (1, 45)	0.465026284587
  (1, 30)	0.465026284587
  (2, 19)	0.367426480502
  (2, 51)	0.465026284587
  (2, 62)	0.465026284587
  (2, 13)	0.465026284587
  (2, 61)	0.465026284587
  (3, 26)	0.408248290464
  (3, 48)	0.408248290464
  (3, 36)	0.408248290464
  (3, 71)	0.408248290464
  (3, 12)	0.408248290464
  (3, 64)	0.408248290464
  (4, 59)	0.353553390593
  (4, 33)	0.353553390593
  (4, 54)	0.353553390593
  :	:
  (7, 21)	0.333333333333
  (7, 2)	0.333333333333
  (7, 65)	0.333333333333
  (7, 37)	0.333333333333
  (7, 50)	0.333333333333
  (8, 29)	0.301511344578
  (8, 68)	0.301511344578
  (8, 28)	0.301511344578
  (8, 0)	0.301511344578
  (8, 17)	0.301511344578
  (8, 69)	0.301511344578
  (8, 57)	0.301511344578
  (8, 60)	0.301511344578
  (8, 34)	0.301511344578
  (8, 67)	0.3015113445

In [912]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/HDUSER/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [913]:
from nltk.tokenize import word_tokenize
import nltk

def process(doc):
    tokens=word_tokenize(doc)
    words=[w.lower() for w in tokens]
    
    porter=nltk.PorterStemmer()
    stemmed_tokens=[porter.stem(t) for t in words]
    
    count=nltk.defaultdict(int)
    for word in words:
        count[word]+=1
    return count

In [914]:
def cos_sim(a,b):
    dot_prod=np.dot(a,b)
    norm_a=np.linalg.norm(a)
    norm_b=np.linalg.norm(b)
    if(norm_a and norm_b):
        return dot_prod/(norm_a*norm_b)
    return dot_prod/(norm_a*norm_a)

In [915]:
def cos_sim1(a,b):
    dot_prod=np.dot(a,b)
    norm_a=np.linalg.norm(a)
    norm_b=np.linalg.norm(b)
    return dot_prod/(norm_a*norm_b)

In [916]:
def get_similarity(dict1,dict2):
    all_words_list=[]
    for key in dict1:
        all_words_list.append(key)
    for key in dict2:
        all_words_list.append(key)
    all_words_list_size=len(all_words_list)
    v1=np.zeros(all_words_list_size,dtype=np.int)
    v2=np.zeros(all_words_list_size,dtype=np.int)
    i=0
    for key in all_words_list:
        v1[i]=dict1.get(key,0)
        v2[i]=dict2.get(key,0)
        i=i+1
    return cos_sim(v1,v2)

In [917]:
def get_similarity1(dict1,dict2):
    all_words_list=[]
    for key in dict1:
        all_words_list.append(key)
    for key in dict2:
        all_words_list.append(key)
    all_words_list_size=len(all_words_list)
    v1=np.zeros(all_words_list_size,dtype=np.int)
    v2=np.zeros(all_words_list_size,dtype=np.int)
    i=0
    for key in all_words_list:
        v1[i]=dict1.get(key,0)
        v2[i]=dict2.get(key,0)
        i=i+1
    return cos_sim1(v1,v2)

In [918]:
t1[2]

'pollution delhi smoking cigarettes smog'

In [919]:
t2[0]

"to encourage use of public transport during odd even delhi govt to allow free travel for commuters in all dtc and cluster buses from 13-17 november ngt asked dtc on the number of its unused buses in its workshops the national green tribunal asks cpcb dpcc and delhi govt why didn't you implement it earlier nothing has been done from your delhi govt end since the past one year safar has forecast increase in the concentration of fine particles of pollutants on thursday madhavan rajeevan secretary ministry of earth sciences tweeted delhi smog is not localised as seen in satellite image i call for cost effective measures since management of air pollution requires sustained actions over a long period of time to be effective"

In [920]:
x=process(t1[2])

In [921]:
y=process(t2[0])

In [922]:
print("Similarity between tweet1 and news1:",get_similarity1(x,y))

Similarity between tweet1 and news1: 0.266732534685


In [923]:
def comp_description(query, results_number=20):
        results=[]
        q_vector = vectorizer.transform([query])
        print("Comparable Description: ", query)
        results.append(cosine_similarity(q_vector, doc_vectors.toarray()))
        f=0
        elem_list=[]
        for i in results[:10]:
            for elem in i[0]:
                    print("Review",f, "Similarity: ", elem)
                    elem_list.append(elem)
                    f+=1
            print("The Review Most similar to the Comparable Description is Description #" ,elem_list.index(max(elem_list)))
            print("Similarity: ", max(elem_list))
            if sum(elem_list) / len(elem_list)==0.0:
                print("No similar descriptions")
            else:
                print(news['news_summary'].loc[elem_list.index(max(elem_list)):elem_list.index(max(elem_list))])

In [924]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import cross_val_score
from sklearn.metrics.pairwise import euclidean_distances
pd.set_option('display.max_colwidth', 1500)

In [925]:
vectorizer = TfidfVectorizer(stop_words='english',
                     binary=False,
                     max_df=0.95, 
                     min_df=0.15,
                     ngram_range = (1,2),use_idf = False, norm = None)
doc_vectors = vectorizer.fit_transform(news['news_summary'])
print(doc_vectors.shape)
print(vectorizer.get_feature_names())

(20, 14)
['15', 'delhi', 'expected', 'including', 'india', 'just', 'long', 'market', 'new', 'rs', 'said', 'time', 've', 'world']


In [926]:
comp_description(users['summary'][2])

Comparable Description:  Pollution Delhi Smoking Cigarettes Smog
Review 0 Similarity:  0.942809041582
Review 1 Similarity:  0.0
Review 2 Similarity:  0.0
Review 3 Similarity:  0.0
Review 4 Similarity:  0.0
Review 5 Similarity:  0.0
Review 6 Similarity:  0.0
Review 7 Similarity:  0.258198889747
Review 8 Similarity:  0.235702260396
Review 9 Similarity:  0.0
Review 10 Similarity:  0.0
Review 11 Similarity:  0.0
Review 12 Similarity:  0.0
Review 13 Similarity:  0.0
Review 14 Similarity:  0.0
Review 15 Similarity:  0.0
Review 16 Similarity:  0.57735026919
Review 17 Similarity:  0.0
Review 18 Similarity:  0.0
Review 19 Similarity:  0.0
The Review Most similar to the Comparable Description is Description # 0
Similarity:  0.942809041582
0    To encourage use of public transport during Odd Even Delhi govt to allow free travel for commuters in all DTC and Cluster buses from 13-17 November NGT asked DTC on the number of its unused buses in its workshops The National Green Tribunal asks CPCB DPCC 

In [927]:
print(t1)

0                                               president trump visit china success us
1                                                     earthquake delhi 8.5 north india
2                                              pollution delhi smoking cigarettes smog
3                                           halloween party kids usa chocolates sweets
4                             rohtak jail prisoner out on bail|prisoners freely lodged
5                                 pm narendra modi meets new zealand pm jacinda ardern
6    properties dawoodibrahim relatives auctioned imc chamber commerce industry mumbai
7                           court bihar bettiah fir farooq abdullah treason lawyer pok
8     incometax unaccounted income 1400 crore unearthed raids sasikala jaya tv chennai
9                         congress power mann ki baat government rahul gandhi visnagar
Name: summary, dtype: object


In [928]:
print(x)

defaultdict(<class 'int'>, {'pollution': 1, 'delhi': 1, 'smoking': 1, 'cigarettes': 1, 'smog': 1})


In [929]:
print(y)

defaultdict(<class 'int'>, {'to': 3, 'encourage': 1, 'use': 1, 'of': 7, 'public': 1, 'transport': 1, 'during': 1, 'odd': 1, 'even': 1, 'delhi': 4, 'govt': 3, 'allow': 1, 'free': 1, 'travel': 1, 'for': 2, 'commuters': 1, 'in': 4, 'all': 1, 'dtc': 2, 'and': 2, 'cluster': 1, 'buses': 2, 'from': 2, '13-17': 1, 'november': 1, 'ngt': 1, 'asked': 1, 'on': 2, 'the': 4, 'number': 1, 'its': 2, 'unused': 1, 'workshops': 1, 'national': 1, 'green': 1, 'tribunal': 1, 'asks': 1, 'cpcb': 1, 'dpcc': 1, 'why': 1, 'did': 1, "n't": 1, 'you': 1, 'implement': 1, 'it': 1, 'earlier': 1, 'nothing': 1, 'has': 2, 'been': 1, 'done': 1, 'your': 1, 'end': 1, 'since': 2, 'past': 1, 'one': 1, 'year': 1, 'safar': 1, 'forecast': 1, 'increase': 1, 'concentration': 1, 'fine': 1, 'particles': 1, 'pollutants': 1, 'thursday': 1, 'madhavan': 1, 'rajeevan': 1, 'secretary': 1, 'ministry': 1, 'earth': 1, 'sciences': 1, 'tweeted': 1, 'smog': 1, 'is': 1, 'not': 1, 'localised': 1, 'as': 1, 'seen': 1, 'satellite': 1, 'image': 1, 'i

In [930]:
similarity=[]
i=0
for a in t1:
    for b in t2:
        a1=process(a)
        b1=process(b)
        x=get_similarity1(a1,b1)
        print("Similarity between "+a+" AND "+b+" is ",x)
        similarity.append(x)
    i=i+1

Similarity between president trump visit china success us AND to encourage use of public transport during odd even delhi govt to allow free travel for commuters in all dtc and cluster buses from 13-17 november ngt asked dtc on the number of its unused buses in its workshops the national green tribunal asks cpcb dpcc and delhi govt why didn't you implement it earlier nothing has been done from your delhi govt end since the past one year safar has forecast increase in the concentration of fine particles of pollutants on thursday madhavan rajeevan secretary ministry of earth sciences tweeted delhi smog is not localised as seen in satellite image i call for cost effective measures since management of air pollution requires sustained actions over a long period of time to be effective is  0.0
Similarity between president trump visit china success us AND on the anniversary of his surprise victory in the 2016 presidential election us president donald trump on wednesday reached beijing on his f

Similarity between earthquake delhi 8.5 north india AND congress vice-president rahul gandhi continued his offensive on prime minister narendra modi during his poll campaigning in central gujarat on monday in a veiled attack on the prime minister’s monthly radio talk mann ki baat the congress scion told the gathering at kheda district we the congress will not tell you our mann ki baat we will listen to your mann ki baat on his three-day journey of several districts in central gujarat titled yuva rozgaar and khedut adhikar yatra as part of navsarjan yatra rahul attacked on the government’s employment generation claims the prime minister’s focus is only 10-15 industrialists it is not those 10-15 industrialists who will create jobs but small and medium businesses rahul told a gathering at kheda huge crowds including farmers women and youth gathered to listen to the congress leader who said progress is possible only if the government removes farmers debts prime minister narendra modi’s guj

In [931]:
k=len(news.index)
print(k)

20


In [932]:
similarity

[0.0,
 0.4677234140250241,
 0.075592894601845442,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.032960068346068586,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.20614722881202929,
 0.13443321448446624,
 0.17118419700436519,
 0.0,
 0.0,
 0.0,
 0.15614401167176531,
 0.092524200411845003,
 0.17313746213992201,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.10206207261596577,
 0.0,
 0.089622142989644174,
 0.0,
 0.0,
 0.0,
 0.26673253468463221,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.050062617432175889,
 0.047457899787624949,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.089622142989644174,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.038984058779272517,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.086246163647086788,
 0.1448334524249516,
 0.071066905451870152,
 0.034380708208626445,
 0.15575643783866591,
 0.038683923545710631,
 0.034847224899287779,
 0.0,
 0.03874921291460643,
 0.069843029576957816,
 0.0,
 0.0,
 0.0,
 0.0,
 0.083333333333333329,
 

In [933]:
i=0
credibility=[]
j=0
while j<len(similarity):
    f=0
    tot=similarity[j]
    while (f<k):
        tot=max(tot,similarity[j])
        j=j+1
        f=f+1
    credibility.append(tot)
    total=0

In [934]:
for m in credibility:
    print(m)

0.467723414025
0.206147228812
0.266732534685
0.0389840587793
0.155756437839
0.173967862406
0.106904496765
0.0326164036527
0.378254819028
0.42916625388


In [935]:
i=1
print(t1[i-1]==t1[i])
print(len(users.index))

False
10


In [936]:
user=[]
i=1
m=credibility[0]
n=0
j=1
while (i<len(t1.index)):
    if(t1[i-1]==t1[i]):
        m+=credibility[j]
        j+=1
        n+=1
    else:
        if(m>0):
            if(n>0):
                m=m/n
            user.append(m)
            m=0
            n=0
        m=credibility[j]
        j+=1
    i+=1
    
if(m>0):
    if(n>0):
        m=m/n
    user.append(m)

In [937]:
for m in user:
    print(m)

0.467723414025
0.206147228812
0.266732534685
0.0389840587793
0.155756437839
0.173967862406
0.106904496765
0.0326164036527
0.378254819028
0.42916625388


In [938]:
cred=[]
for m in user:
    if (m>=0.65):
        cred.append("Highly Credible")
    elif(m>=0.40):
        cred.append("Credible")
    elif(m>=0.20):
        cred.append("Not Credible")
    else:
        cred.append("Highly Uncredible")

In [939]:
for m in cred:
    print(m)

Credible
Not Credible
Not Credible
Highly Uncredible
Highly Uncredible
Highly Uncredible
Highly Uncredible
Highly Uncredible
Not Credible
Credible


In [940]:
from astropy.table import Table

In [941]:
t = Table([users['user_id'], cred], names=('User', 'Credibility'), meta={'name': 'first table'})


In [942]:
print(t)

 User     Credibility   
------ -----------------
xyz123          Credible
  yudi      Not Credible
abc123      Not Credible
 alpha Highly Uncredible
  beta Highly Uncredible
  yudi Highly Uncredible
abc123 Highly Uncredible
 alpha Highly Uncredible
 alpha      Not Credible
xyz123          Credible


In [1068]:
t.show_in_notebook()

idx,User,Credibility
0,xyz123,Credible
1,yudi,Not Credible
2,abc123,Not Credible
3,alpha,Highly Uncredible
4,beta,Highly Uncredible


In [943]:
similarity1=[]
credibility1=[]
cred1=[]

In [944]:
import random
import string
import os


class SimilarityReporter:


    def __init__(self, base_corpus, input_corpus):

        self.base_words = self.make_words_list(base_corpus)
        self.input_words = self.make_words_list(input_corpus)
        
        self.base_doubles = self.loadDoubles(self.base_words)
        #for i in self.base_doubles:
         #   print(i)
        self.input_doubles = self.loadDoubles(self.input_words)
        #for i in self.input_doubles:
         #   print(i)

        self.base_triples = self.loadTriples(self.base_words)
        self.input_triples = self.loadTriples(self.input_words)
        
      #  for i in self.base_triples:
       #     print(i)
        
       # for i in self.input_triples:
        #    print(i)

        self.stop_words = ['a', 'an', 'and', 'are', 'as', 'at', 'be', 'by  for', 'from', 'has', 'he', 'in', 'is', 'it', 'its','of', 'on', 'that', 'the', 'to', 'was', 'were','will', 'with']

    def make_words_list(self, corpus):
        words = []
        for word in corpus.split():
            words.append(''.join(char for char in word if char not in string.punctuation).strip())
        
        return words

    def loadDoubles(self, corpus):

        pairs = {}
        for i in range(len(corpus) - 1):
            first, second = corpus[i], corpus[i+1]
            key = first
            if key in pairs:
                pairs[key].append(second)
            else:
                pairs[key] = [second]
        return pairs

    def loadTriples(self, corpus):

        triples = {}
        for i in range(len(corpus) - 2):
            first, second, third = corpus[i], corpus[i+1], corpus[i+2]
            key = (first, second)
            
            if key in triples:
                triples[key].append(third)
            else:
                triples[key] = [third]

        return triples

    def getSimilarityIndexSingles(self):

        matches = 0
        attempts = 0

        for word in self.base_words:
            if word not in self.stop_words:
                if word in self.input_words:
                    matches+=1
            attempts += 1

        print('Single word matches: {}/{} matches, {} match percentage'.format(matches, attempts,round(((float(matches) / attempts) * 100), 3)))

        return matches / attempts

    def getSimilarityIndexDoubles(self):
        matches = 0
        attempts = 0

        for key in self.input_doubles.keys():
            if key in self.base_doubles.keys():
                
                for next_word in self.input_doubles[key]:
                    
                    if next_word in self.base_doubles[key]:
                        matches += 1
            attempts += 1


        print('Two-word sequence similarity: {}/{} matches, {} match percentage'.format(matches, attempts,round(((float(matches) / attempts) * 100), 3)))

        return matches / attempts

    
    def getSimilarityIndexTriples(self):

        matches = 0
        attempts = 0

        for key in self.input_triples.keys():
            if key in self.base_triples.keys():
                
                for next_word in self.input_triples[key]:
                    if next_word in self.base_triples[key]:
                        matches += 1
            attempts += 1

        print('Three-word sequence similarity: {}/{} matches, {} match percentage'.format(matches, attempts,round(((float(matches) / attempts) * 100), 3)))

        return matches / attempts


    def printReport(self):

        print('Generating similarity indexes.....\n')
        
        similarityIndex = 3 * self.getSimilarityIndexSingles() +4 * self.getSimilarityIndexDoubles() +5 * self.getSimilarityIndexTriples()
        similarityIndex=similarityIndex/12
        #similarityIndex=self.getSimilarityIndexSingles()
        similarity1.append(similarityIndex)




In [945]:
for i in t1:
    for j in t2:
        mcc=SimilarityReporter(i,j)
        mcc.printReport()

Generating similarity indexes.....

Single word matches: 0/6 matches, 0.0 match percentage
Two-word sequence similarity: 0/95 matches, 0.0 match percentage
Three-word sequence similarity: 0/120 matches, 0.0 match percentage
Generating similarity indexes.....

Single word matches: 5/6 matches, 83.333 match percentage
Two-word sequence similarity: 1/85 matches, 1.176 match percentage
Three-word sequence similarity: 0/124 matches, 0.0 match percentage
Generating similarity indexes.....

Single word matches: 2/6 matches, 33.333 match percentage
Two-word sequence similarity: 0/76 matches, 0.0 match percentage
Three-word sequence similarity: 0/112 matches, 0.0 match percentage
Generating similarity indexes.....

Single word matches: 0/6 matches, 0.0 match percentage
Two-word sequence similarity: 0/105 matches, 0.0 match percentage
Three-word sequence similarity: 0/141 matches, 0.0 match percentage
Generating similarity indexes.....

Single word matches: 0/6 matches, 0.0 match percentage
Two-

In [946]:
for m in similarity1:
    print(m)

0.0
0.2122549019607843
0.08333333333333333
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.041666666666666664
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.05000000000000001
0.05000000000000001
0.05000000000000001
0.0
0.0
0.0
0.10000000000000002
0.10000000000000002
0.10000000000000002
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.15
0.0
0.0
0.0
0.0
0.0
0.0
0.05000000000000001
0.05000000000000001
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.041666666666666664
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.03125
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.027777777777777776
0.06871345029239766
0.0
0.0588235294117647
0.027777777777777776
0.0
0.027777777777777776
0.027777777777777776
0.027777777777777776
0.027777777777777776
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.027777777777777776
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.027777777777777776
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

In [947]:
i=0
j=0
while j<len(similarity1):
    f=0
    tot=similarity1[j]
    while(f<k):
        tot=max(tot,similarity1[j])
        j=j+1
        f=f+1
    credibility1.append(tot)
    total=0


In [948]:
for m in credibility1:
    print(m)

0.2122549019607843
0.10000000000000002
0.15
0.041666666666666664
0.03125
0.06871345029239766
0.027777777777777776
0.027777777777777776
0.14588744588744587
0.22600081699346405


In [949]:
user1=[]
i=1
m=credibility1[0]
n=0
j=1
while (i<len(t1.index)):
    if(t1[i-1]==t1[i]):
        m+=credibility1[j]
        j+=1
        n+=1
    else:
        if(m>0):
            if(n>0):
                m=m/n
            user1.append(m)
            m=0
            n=0
        m=credibility1[j]
        j+=1
    i+=1
    
if(m>0):
    if(n>0):
        m=m/n
    user1.append(m)

In [950]:

for m in user1:
    if (m>=0.65):
        cred1.append("Highly Credible")
    elif(m>=0.35):
        cred1.append("Credible")
    elif(m>=0.15):
        cred1.append("Not Credible")
    else:
        cred1.append("Highly Uncredible")


In [951]:
t1 = Table([users['user_id'], cred1], names=('User', 'Credibility'), meta={'name': 'second table'})


In [952]:
print(t1)

 User     Credibility   
------ -----------------
xyz123      Not Credible
  yudi Highly Uncredible
abc123      Not Credible
 alpha Highly Uncredible
  beta Highly Uncredible
  yudi Highly Uncredible
abc123 Highly Uncredible
 alpha Highly Uncredible
 alpha Highly Uncredible
xyz123      Not Credible


# Credibility on the basis of profile

In [953]:
cred_profile=[]
for i in t1:
    if

SyntaxError: invalid syntax (<ipython-input-953-12fd6dacdb2c>, line 3)

In [954]:
import http.client
from urllib import parse

def unshorten_url(url):
    parsed = parse(url)
    h = httplib.HTTPConnection(parsed.netloc)
    h.request('HEAD', parsed.path)
    response = h.getresponse()
    if response.status/100 == 3 and response.getheader('Location'):
        return response.getheader('Location')
    else:
        return url

In [955]:
# urllib2
import urllib
import urllib.request as urlreq
link = urlreq.urlopen("http://www.google.com")
#fp = url('http://bit.ly/rgCbf')
link.geturl()

# ==> 'http://webdesignledger.com/freebies/the-best-social-media-icons-all-in-one-place'

#############

# httplib
import http.client
conn=http.client.HTTPConnection('bit.ly')
conn.request('HEAD', '/rgCbf')
response = conn.getresponse()
response.getheader('location')
# ==> 'http://webdesignledger.com/freebies/the-best-social-media-icons-all-in-one-place'

#############

# pycurl
def url_resolve(url):
    import pycurl
    conn = pycurl.Curl()
    conn.setopt(pycurl.URL, url)
    conn.setopt(pycurl.FOLLOWLOCATION, 1)
    conn.setopt(pycurl.CUSTOMREQUEST, 'HEAD')
    conn.setopt(pycurl.NOBODY, True)
    conn.perform()
    x=conn.getinfo(pycurl.EFFECTIVE_URL)
    print(x)

In [737]:
url="http://bit.ly/rjjjjjjjjkjkgCbf"
url_resolve(url)

KeyboardInterrupt: 

In [852]:
import urllib
from urllib import parse
 
def get_pagerank(url):
    hsh = check_hash(hash_url(url))
    gurl = 'http://www.google.com/search?client=navclient-auto&features=Rank:&q=info:%s&ch=%s' % (urllib.parse.quote(url), hsh)
    print(gurl)
    try:
        f = urllib.request.urlopen(gurl)
        rank = f.read().strip()[9:]
        print(rank)
    except Exception:
        rank = 'N/A'
    if rank == '':
        rank = '0'
    return rank
 
 
def  int_str(string, integer, factor):
    for i in range(len(string)) :
        integer *= factor
        integer &= 0xFFFFFFFF
        integer += ord(string[i])
    return integer
 
 
def hash_url(string):
    c1 = int_str(string, 0x1505, 0x21)
    c2 = int_str(string, 0, 0x1003F)
 
    c1 >>= 2
    c1 = ((c1 >> 4) & 0x3FFFFC0) | (c1 & 0x3F)
    c1 = ((c1 >> 4) & 0x3FFC00) | (c1 & 0x3FF)
    c1 = ((c1 >> 4) & 0x3C000) | (c1 & 0x3FFF)
 
    t1 = (c1 & 0x3C0) << 4
    t1 |= c1 & 0x3C
    t1 = (t1 << 2) | (c2 & 0xF0F)
 
    t2 = (c1 & 0xFFFFC000) << 4
    t2 |= c1 & 0x3C00
    t2 = (t2 << 0xA) | (c2 & 0xF0F0000)
 
    return (t1 | t2)
 
 
def check_hash(hash_int):
    hash_str = '%u' % (hash_int)
    flag = 0
    check_byte = 0
 
    i = len(hash_str) - 1
    while i >= 0:
        byte = int(hash_str[i])
        if 1 == (flag % 2):
            byte *= 2;
            byte = byte / 10 + byte % 10
        check_byte += byte
        flag += 1
        i -= 1
 
    check_byte %= 10
    if 0 != check_byte:
        check_byte = 10 - check_byte
        if 1 == flag % 2:
            if 1 == check_byte % 2:
                check_byte += 9
            check_byte=int(check_byte)
            check_byte >>= 1
 
    return '7' + str(check_byte) + hash_str

In [853]:
import urllib.request
with urllib.request.urlopen("http://www.python.org") as url:
    s = url.read()

In [740]:
url = "http://www.google.com/"
print(get_pagerank(url))

http://www.google.com/search?client=navclient-auto&features=Rank:&q=info:http%3A//www.google.com/&ch=72804486762
N/A


In [956]:
prof=pd.read_csv('/home/HDUSER/profile.csv')

In [957]:
prof['verified'][0]

True

In [958]:
points=[]
i=0
while (i<len(prof.index)):
    point=0
    if(prof['verified'][i]==True):
        point+=2
    ratio=prof['followers'][i]/prof['following'][i]
    if(ratio>0.8):
        point+=2
    elif(ratio>0.4):
        point+=1
    if(prof['linked'][i]==True):
        point+=1
    if(len(str(prof['website'][i]))>5):
        point+=1
    if(len(str(prof['description'][i]))>5):
        point+=1
    points.append(point)
    i+=1

In [959]:
for m in points:
    print(m)

6
3
1
1
7


In [960]:
import re
i=1
point=0

if(users['retweets'][i-1]>75):
    point+=1
if(users['favorites'][i-1]>75):
    point+=1
points[i-1]+=point
while (i<len(points)):
    if(users['user_id'][i-1]!=users['user_id'][i]):
        point=0
    if(users['retweets'][i]>75):
        point+=1
    if(users['favorites'][i]>75):
        point+=1
    if(re.search("(?P<url>https?://[^\s]+)", users['tweet'][i])==1):
        url=re.search("(?P<url>https?://[^\s]+)", users['tweet'][i]).group("url")
        if(url_resolve(url)):
            point+=1
    points[i]+=point
    i+=1

In [961]:
for m in points:
    print(m)

7
3
1
1
7


In [962]:
i=0
while (i<len(points)):
    print("Before",credibility[i])
    points[i]=points[i]/10
    credibility[i]=credibility[i]*0.75+0.25*points[i]
    print(credibility[i])
    i+=1

Before 0.467723414025
0.525792560519
Before 0.206147228812
0.229610421609
Before 0.266732534685
0.225049401013
Before 0.0389840587793
0.0542380440845
Before 0.155756437839
0.291817328379


In [963]:
cred_matrix=[]
for m in user:
    if (m>=0.65):
        cred_matrix.append("Highly Credible")
    elif(m>=0.40):
        cred_matrix.append("Credible")
    elif(m>=0.20):
        cred_matrix.append("Not Credible")
    else:
        cred_matrix.append("Highly Uncredible")

In [964]:
t8 = Table([users['user_id'], cred_matrix], names=('User', 'Credibility'), meta={'name': 'second table'})

In [965]:
t8

User,Credibility
object,str17
xyz123,Credible
yudi,Not Credible
abc123,Not Credible
alpha,Highly Uncredible
beta,Highly Uncredible
yudi,Highly Uncredible
abc123,Highly Uncredible
alpha,Highly Uncredible
alpha,Not Credible


In [974]:
t8

User,Credibility
object,str17
xyz123,Credible
yudi,Not Credible
abc123,Not Credible
alpha,Highly Uncredible
beta,Highly Uncredible


# Building User Profiles based on their interests

In [503]:
data=[]

In [975]:
text_files=['/home/HDUSER/data/sports','/home/HDUSER/data/entertainment','/home/HDUSER/data/health','/home/HDUSER/data/lifestyle','/home/HDUSER/data/politics','/home/HDUSER/data/relationships','/home/HDUSER/data/technology']

In [976]:
data = [open(f,'r') for f in text_files]

In [977]:
for m in data:
    print(m.name)

/home/HDUSER/data/sports
/home/HDUSER/data/entertainment
/home/HDUSER/data/health
/home/HDUSER/data/lifestyle
/home/HDUSER/data/politics
/home/HDUSER/data/relationships
/home/HDUSER/data/technology


In [978]:
sports=""
entertainment=""
health=""
lifestyle=""
politics=""
relationships=""
technology=""
sports = open(text_files[0], 'r').read()
entertainment=open(text_files[1], 'r').read()
health=open(text_files[2], 'r').read()
lifestyle=open(text_files[3], 'r').read()
politics=open(text_files[4], 'r').read()
relationships=open(text_files[5], 'r').read()
technology=open(text_files[6], 'r').read()

In [979]:
print(cricket)

PROMISING MITHUN

Mithun breathes promise into Karnataka's Ranji aspirations

• Last updated on Sun, 12 Nov, 2017, 07:03 PM Akshay Maanay in Alur

Abhimanyu Mithun picked up four wickets in a six-over spell to help Karnataka gain a crucial first-innings lead. © Getty

Abhimanyu Mithun's hands pointed skywards and the head fell back in relief after he scalped Delhi's last wicket - Kulwant Khejroliya. The right-arm pacer had picked up a five-fer for the second time this season, making this his best start to a Ranji season since Karnataka's eventful season in 2014-15.

He and Vinay Kumar have shouldered Karnataka's fast bowling responsibilities since he made his debut in 2009-10. The duo have now formed a famed partnership around which the state's other fast bowlers revolve. Though Mithun is more injury prone than his skipper, but when fit, he is a vital cog in the Karnataka team and that was on display when he accumulated 80 wickets across two title-winning campaigns.

The last two years

In [980]:
category=["Sports","Entertainment","Health","Lifestyle","Politics","Relationships","Technology"]

In [981]:
categories=[sports,entertainment,health,lifestyle,politics,relationships,technology]

In [984]:
t1=users['summary']
t1=t1.str.lower()

In [985]:
t1[0]

'president trump visit china success us'

In [986]:
x=process(t1[0])

In [987]:
y=process(categories[0])

In [988]:
get_similarity(x,y)

0.0065219625947017168

In [989]:
y=process(health)

In [990]:
get_similarity(x,y)

0.012671177704957758

In [991]:
y=process(relationships)

In [992]:
get_similarity(x,y)

0.0

In [534]:
#t1.add_getuserlikes()
#t1.add_getuserRTS()

In [1053]:
j=0
t4 = Table(names=('User', 'Cricket', 'Celebs','Health','Lifestyle','Politics','Relationships','Technology'), dtype=('S10','S10', 'S10', 'S10','S10', 'S10', 'S10','S10'))
for x in t1:
    i=0
    user_p=[users['user_id'][j]]
    print("\n\n\n\nUSER  --> ",users['user_id'][j],"\n------------------------------")

    x=process(x)
    for y in categories:
        y=process(y)
        sim=get_similarity(x,y)
        print(sim)
        user_p.append(sim)
        i+=1
    j+=1
    t4.add_row(user_p)





USER  -->  xyz123 
------------------------------
0.0065219625947
0.0
0.012671177705
0.0
0.0299275135631
0.0
0.0




USER  -->  yudi 
------------------------------
0.0211272844561
0.0
0.0
0.0
0.0
0.0
0.0




USER  -->  abc123 
------------------------------
0.0211272844561
0.0
0.0
0.0
0.0
0.0
0.0




USER  -->  alpha 
------------------------------
0.0
0.0
0.0
0.0
0.0299275135631
0.0
0.0




USER  -->  beta 
------------------------------
0.102943612467
0.0689655172414
0.0777022705906
0.0122036892295
0.0657141435573
0.0
0.103720519717




USER  -->  yudi 
------------------------------
0.0129411366671
0.0399361531915
0.00967777182953
0.0105687048926
0.0
0.0
0.00868908445985




USER  -->  abc123 
------------------------------
0.0
0.0
0.0
0.0
0.0
0.0
0.0




USER  -->  alpha 
------------------------------
0.0
0.0
0.0
0.0
0.0125269423258
0.0
0.0




USER  -->  alpha 
------------------------------
0.0
0.0200040012004
0.0
0.0
0.0
0.0
0.0




USER  -->  xyz123 
---------------------

In [1054]:
t4.sort('User')
print(t4)

 User   Cricket     Celebs     Health   ...  Politics  Relationships Technology
------ ---------- ---------- ---------- ... ---------- ------------- ----------
abc123 0.02112728        0.0        0.0 ...        0.0           0.0        0.0
abc123        0.0        0.0        0.0 ...        0.0           0.0        0.0
 alpha        0.0        0.0        0.0 ... 0.02992751           0.0        0.0
 alpha        0.0        0.0        0.0 ... 0.01252694           0.0        0.0
 alpha        0.0 0.02000400        0.0 ...        0.0           0.0        0.0
  beta 0.10294361 0.06896551 0.07770227 ... 0.06571414           0.0 0.10372051
xyz123 0.00652196        0.0 0.01267117 ... 0.02992751           0.0        0.0
xyz123 0.00545666        0.0        0.0 ... 0.17756082           0.0 0.00951841
  yudi 0.02112728        0.0        0.0 ...        0.0           0.0        0.0
  yudi 0.01294113 0.03993615 0.00967777 ...        0.0           0.0 0.00868908


In [1055]:
i=0
while i<len(t4)-1:
    if(t4[i][0]==t4[i+1][0]):
        j=1
        while j<8:
            t4[i][j]+=t4[i+1][j]
            j+=1
        t4.remove_row(i+1)
    i+=1
            

/home/HDUSER/anaconda3/lib/python3.6/site-packages/astropy/table/row.py:48: StringTruncateWarning: truncated right side string(s) longer than 10 character(s) during assignment
  self._table.columns[item][self._index] = val
/home/HDUSER/anaconda3/lib/python3.6/site-packages/astropy/table/row.py:48: StringTruncateWarning: truncated right side string(s) longer than 10 character(s) during assignment
  self._table.columns[item][self._index] = val
/home/HDUSER/anaconda3/lib/python3.6/site-packages/astropy/table/row.py:48: StringTruncateWarning: truncated right side string(s) longer than 10 character(s) during assignment
  self._table.columns[item][self._index] = val
/home/HDUSER/anaconda3/lib/python3.6/site-packages/astropy/table/row.py:48: StringTruncateWarning: truncated right side string(s) longer than 10 character(s) during assignment
  self._table.columns[item][self._index] = val
/home/HDUSER/anaconda3/lib/python3.6/site-packages/astropy/table/row.py:48: StringTruncateWarning: truncated

In [1056]:
print(t4)

 User   Cricket     Celebs     Health   ...  Politics  Relationships Technology
------ ---------- ---------- ---------- ... ---------- ------------- ----------
abc123 0.02112728     0.00.0     0.00.0 ...     0.00.0        0.00.0     0.00.0
 alpha     0.00.0     0.00.0     0.00.0 ... 0.02992751        0.00.0     0.00.0
 alpha        0.0 0.02000400        0.0 ...        0.0           0.0        0.0
  beta 0.10294361 0.06896551 0.07770227 ... 0.06571414           0.0 0.10372051
xyz123 0.00652196     0.00.0 0.01267117 ... 0.02992751        0.00.0 0.00.00951
  yudi 0.02112728 0.00.03993 0.00.00967 ...     0.00.0        0.00.0 0.00.00868


In [1057]:
i=0
t3 = Table(names=('User', 'Interest'), dtype=('S10','S15'))
while i<len(t4):
    j=1
    user_p=[users['user_id'][i]]
    maxval=t4[i][j]
    maxindex=j
    while j<8:
        if(t4[i][j]>maxval):
            maxval=t4[i][j]
            maxindex=j
        j+=1
    user_p.append(category[maxindex-1])
    t3.add_row(user_p)
    i+=1
    
print('\n\n\n\n',t3)





  User     Interest  
------ -------------
xyz123        Sports
  yudi      Politics
abc123 Entertainment
 alpha    Technology
  beta      Politics
  yudi        Sports


In [1070]:
t3.show_in_notebook()

idx,User,Interest
0,xyz123,Sports
1,yudi,Politics
2,abc123,Entertainment
3,alpha,Technology
4,beta,Politics


In [1060]:
import csv
from astropy.io import ascii
ascii.write(t3, 'user_profile.csv', format='csv', fast_writer=False)